In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import pandas as pd
import lightgbm as lgb

In [ ]:
DEBUG = True

In [ ]:
#读取数据集
train_df = pd.read_csv('/kaggle/input/news01/train_set.csv',sep='\t')
test_df = pd.read_csv('/kaggle/input/news01/test_a.csv',sep='\t')
if DEBUG:
    train_df = train_df[:100]

In [ ]:
#Tfidf编码
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=1000).fit(train_df['text'].iloc[:].values)
train_tfidf = tfidf.transform(train_df['text'].iloc[:].values)
test_tfidf = tfidf.transform(test_df['text'].iloc[:].values)
print('Finish Tfidf')

#Lgb
# clf = RidgeClassifier()
# clf = LogisticRegression()
clf = LGBMClassifier()
clf.fit(train_tfidf, train_df['label'].iloc[:].values)
print('Finish Lgb')

In [ ]:
res = pd.DataFrame()
res['label'] = clf.predict(test_tfidf)
res.to_csv('base_submission.csv', index=None)

In [ ]:
n_fold = 5
label = 'label'
#lgb参数
lgb_params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    'metric': 'multi_error',
    "num_class": 14,
    "learning_rate": 0.01,
    "max_depth": -1,
    "num_leaves": 15,
    "feature_fraction": 1,
    "bagging_fraction": 1,     
    'min_data_in_leaf': 100,  
    'bagging_freq': 6, 
    "nthread":-1,
    "verbose":-1,
    
}
y_val = np.zeros((train_tfidf.shape[0],14))
y_test = np.zeros((test_tfidf.shape[0],14))

skf = StratifiedKFold(n_splits=n_fold,shuffle=True,random_state=42)
for train_index,valid_index in skf.split(train_df,train_df[label]):
    
    x_train,x_valid,y_train,y_valid = train_tfidf[train_index],train_tfidf[valid_index],train_df[label].iloc[train_index],train_df[label].iloc[valid_index]

    train_data = lgb.Dataset(x_train,label=y_train)
    valid_data = lgb.Dataset(x_valid,label=y_valid)
    
    model = lgb.train(lgb_params,train_data,valid_sets=[train_data,valid_data],num_boost_round=10000,verbose_eval=50, early_stopping_rounds = 50)
    
    y_val[valid_index] = model.predict(x_valid)
    y_test += np.array(model.predict(test_tfidf)/n_fold)

cv_score = f1_score(np.argmax(y_val,axis=1),train_df[label].values, average='macro')
print(f'local cv: {cv_score}')

In [ ]:
res = pd.DataFrame()
res['label'] = np.argmax(y_test,axis=1)
res.to_csv('c_submission.csv', index=None)